In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'torch'

In [8]:
import pandas as pd
data = pd.read_csv("./WebKB/content/cornell.content", header=None, delim_whitespace=True).rename(columns={0:"URL",1704:"class"})

In [9]:
data

,URL,1,2,3,4,5,6,7,8,9,...,1695,1696,1697,1698,1699,1700,1701,1702,1703,class
0,http://cam.cornell.edu/ph/index.html,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,student
1,http://cam.cornell.edu/~baggett/index.html,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,student
2,http://cs-tr.cs.cornell.edu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,project
3,http://cs.cornell.edu/info/courses/current/cs4...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,course
4,http://cs.cornell.edu/info/courses/fall-95/cs4...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,course
5,http://cs.cornell.edu/info/courses/spring-96/c...,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,course
6,http://dri.cornell.edu,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,project
7,http://dri.cornell.edu/pub/people/davis.html,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,staff
8,http://simon.cs.cornell.edu/info/courses/curre...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,course
9,http://simon.cs.cornell.edu/info/courses/sprin...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,course
